# 1. Protein

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [2]:
dataset = 'protein'
load ('./data/protein.mat');
[ndata, ~] = size(M);
X = M(:,2:9);
Y = M(:,1);
R = randperm(ndata);        
xt = X(R(1:5730),:);
yt = Y(R(1:5730));
R(1:5730) = [];
x = X(R,:);          
y = Y(R);
n = 40000; nt = 5730;   %Ms = 50

dataset = protein


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'Y' ;
opts.Ms = 50 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/energymodel', 'models','t_dGP_train')

In [4]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/proteinmodel');
models = s.models;


## 1.2 GRBCM vs TEGRBCM

### 1.2.1 GRBCM baseline

In [5]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing protein kmeans GRBCM
GRBCM: SMSE 5.25166934, MSLL -0.9517, NLPD 2.2785


### 1.2.2 Grid q

In [6]:
criterions = {'TEGRBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:2.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegrbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing protein kmeans TEGRBCM
TEGRBCM: SMSE 4.74397447, MSLL -1.0342, NLPD 2.1961 - q   0.20
TEGRBCM: SMSE 4.67093111, MSLL -1.0630, NLPD 2.1673 - q   0.40
TEGRBCM: SMSE 4.75820090, MSLL -1.0356, NLPD 2.1946 - q   0.60
TEGRBCM: SMSE 4.95647439, MSLL -0.9955, NLPD 2.2348 - q   0.80
TEGRBCM: SMSE 5.25166934, MSLL -0.9517, NLPD 2.2785 - q   1.00
TEGRBCM: SMSE 5.63684472, MSLL -0.9075, NLPD 2.3228 - q   1.20
TEGRBCM: SMSE 6.10599687, MSLL -0.8640, NLPD 2.3662 - q   1.40
TEGRBCM: SMSE 6.65212316, MSLL -0.8221, NLPD 2.4082 - q   1.60
TEGRBCM: SMSE 7.26668297, MSLL -0.7820, NLPD 2.4483 - q   1.80
TEGRBCM: SMSE 7.93967891, MSLL -0.7440, NLPD 2.4863 - q   2.00


### 1.2.3 Find single q

In [7]:
criterion = 'TEGRBCM';

In [ ]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=0.4);

Optimizing q


In [ ]:
q

In [ ]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.2.4 Find multi q

In [ ]:
criterion = 'TEGRBCM';

In [ ]:
printf('Optimizing q\r\n');
iqs = q + 0.01 * rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=5000, lr=1e-8, lambda=1e6, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

In [ ]:
qs
save('./res/tegrbcmenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);